# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

df = pd.read_csv("homepage_actions.csv")
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [3]:
df["action"].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [4]:
view = set(df[df["action"] == "view"]["id"].unique())
click = set(df[df["action"] == "click"]["id"].unique())

print("Number of viewers: {} \tNumber of clickers: {}".format(len(view), len(click)))
print("Number of Viewers who didn't click: {}".format(len(view-click)))
print("Number of Clickers who didn't view: {}".format(len(click-view)))

Number of viewers: 6328 	Number of clickers: 1860
Number of Viewers who didn't click: 4468
Number of Clickers who didn't view: 0


In [5]:
exp = set(df[df.group=='experiment']['id'].unique())
control = set(df[df.group=='control']['id'].unique())
print('Overlap of experiment and control groups: {}'.format(len(exp&control)))

Overlap of experiment and control groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [6]:
#Your code here
df["count"] = 1
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [7]:
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

In [8]:
import scipy.stats as stats

def welch_t(a, b):
    
    num = a.mean() - b.mean()
    
    denom = np.sqrt(a.var(ddof=1)/len(a) + b.var(ddof=1)/len(b))
    
    return np.abs(num/denom)

In [9]:
def welch_df(a, b):
    
    s1 = a.var(ddof=1)
    s2 = b.var(ddof=1)
    n1 = len(a)
    n2 = len(b)
    
    num = (s1/n1 + s2/n2)**2
    denom = ((s1/n1)**2/(n1 - 1)) + ((s2/n2)**2/(n2 - 1))
    
    return num/denom

In [10]:
def p_value(a, b, two_sided=False):
    
    t = welch_t(a, b)
    df = welch_df(a, b)
    
    p = 1-stats.t.cdf(np.abs(t), df)
    
    if two_sided:
        return 2*p
    else:
        return p

In [11]:
p_value(control["click"], experiment["click"])

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [12]:
#Your code here
rate = control["click"].mean()
expected_number_clicks = rate * len(experiment)
print(expected_number_clicks)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [13]:
#Your code here
n = len(experiment)
p = rate
var = n * p * (1-p)
std = np.sqrt(var)
print(std)

24.568547907005815


In [14]:
experiment_clicks = experiment["click"].sum()
z_score = (experiment_clicks - expected_number_clicks)/std
print(z_score)

3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [15]:
#Your code here
p = stats.norm.sf(z_score)
print(p)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [16]:
"""
While the calculated p value is lower than the statistical test p value, both are smaller than the value of alpha, allowing us to reject the null hypothessis. The experiemntal page is more effective.
"""

'\nWhile the calculated p value is lower than the statistical test p value, both are smaller than the value of alpha, allowing us to reject the null hypothessis. The experiemntal page is more effective.\n'

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.